# Automated Cell Tracking Script
### Makes heavy use of the oyLabImaging package produced by Alon Oyler-Yaniv's group, which can be found at https://github.com/oylab/oyLabImaging
#### Author: Woody March-Steinman, Program in Applied Mathematics, University of Arizona.
- Contact: wmarchsteinman@arizona.edu

This code provides a framework for a reproducible automated cell tracking pipeline using existing tools from Alon Oyler-Yaniv's lab.
Included as additional features are the following:
- Streamlined data output for downstream analysis
- Calculation of frame-to-frame property deltas for some properties
- Included analysis pipeline options, including visualization of results.
- Fault tolerance and consistency. For large movies, this will reduce the chance of losing data after long processing times.

Future versions of the code will include:
- *Mitosis prediction, lineage tracking, and track correction* via both CDK2 marker tracking and through other prediction metrics.
- *Death prediction*,  will require use of a standard set of metrics (typically a nuclear marker and brightfield imaging)
- *Track correction*, aiming to use the viterbi algorithm or some variant 

## Imports

Necessary packages to install before use: 
pandas, pyfftw, numpy, matplotlib, oyLabImaging

In [ ]:
%load_ext autoreload
%autoreload 2
%gui qt
%matplotlib qt5

import sys
import os

#import cupy as cpW

import pandas as pd
import pyfftw
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math


#from utilities.py import *
from oyLabImaging import Metadata
from oyLabImaging.Processing import results
from oyLabImaging.Processing.imvisutils import get_or_create_viewer



## Data Preparation and Drift Correction

Note here that you need to have the following files in the directory denoted in $\textbf{fpath}$:
- *movie.nd2* -- Can have any name.  Only one ND2 file per data folder.  Aim to have files with more than one frame.

- *labels.csv* -- This file should contain all experimental conditions and the positions that match. (ex: Control as experiment name, 1 as starting position, 10 as ending position -- actual nd2 positions are assumed to start from 0, but use experimental positions as this is automatically handled)

- *markers.csv* -- This file should contain fluorescent markers identical to the names used in the .nd2 file, and unspaced labels you wish to use to refer to them. (ex: YFP as the marker, FOXO1 as the label)

**NOTE:** The code below automatically replaces spaces with underscores.  If you don't want this behavior, you can delete it, but it might cause downstream issues.


In [ ]:
#------------------- data setup -- edit names to match.  Also run from a fresh notebook before analysis. --------------------#
movie_name = 'k_all' #no spaces, use underscores instead.
fpath = './data/kelvin_all' #directory that nd2 file is stored in.
mpath = './'+movie_name+'_analysis' #do not change, ideally

nframes = 256
nuc_mark = "640S"
cyto_mark = "488S"

#---------------- Don't edit below here -- Metadata loading and label generation -------------------#
"""generate experiment labels and position ranges. 
labels.csv/markers.csv files must be in the fpath folder above, names should contain no periods or spaces.
"""
label_csv = pd.read_csv(fpath + "/labels.csv")
labels = {}
for i in range(len(label_csv)):
    curr_lab = label_csv["experiment_label"][i]
    curr_lab = curr_lab.replace(".","_")
    curr_lab = curr_lab.replace(" ", "_")
    labels.update({curr_lab: np.array(range(label_csv["position_start"][i] - 1, label_csv["position_end"][i]))})

marker_csv = pd.read_csv(fpath + "/markers.csv")
marker_labels = {}
for i in range(len(marker_csv)):
    curr_lab = marker_csv["name"][i]
    curr_lab = curr_lab.replace(".","_")
    curr_lab = curr_lab.replace(" ", "_")
    marker_labels.update({marker_csv["marker"][i]: curr_lab})

#creates filepath for movie analysis.
os.makedirs(mpath, exist_ok=True)


In [ ]:
MD = Metadata(fpath)


In [ ]:
MD()

In [ ]:
#this section deals with 
MD = Metadata(fpath)


#only run this if you haven't yet for this movie.  Otherwise, delete the metadata.pickle file and re-run this.
if np.max(MD().frame) > nframes:
     MD().frame = MD().frame//len(set(MD()["Position"]))
     MD.save()


# #---------------- Don't edit below here -- Drift Correction -- REQUIRES LOTS OF MEMORY-------------------#

#this makes some assumptions regarding position names in the nd2 file.
positions_to_track = []
for i in labels.values():
    positions_to_track = positions_to_track + list(i)
positions_to_track = [str(i) for i in positions_to_track]
if "driftTform" not in MD().columns:
   MD.CalculateDriftCorrection(Channel=nuc_mark, GPU = False)


#---------------- Don't edit above here -------------------#

#view metadata
MD()

#uncomment to ensure metadata pickle file is saved, including drift correction
#MD.save()


# -- uncomment to view movie with drift correction in Napari Viewer --
#MD.viewer() 


In [3]:
from pathlib import Path
foo = Path("data")
bar = "path"
print(foo / bar)

data/path


# Segmentation and Tracking

In [ ]:
#builds either an empty results object or a results object generated from previously stored results data (results.pickle)
R = results(MD=MD)

R()

In [ ]:
#---------------- Segmentation and Tracking -- REQUIRES LOTS OF MEMORY/COMPUTE RESOURCES-------------------#

#you can change the parameters below to the version of your choice if your segmentation runs fast, but these give fair results.  If it runs slow, keep these values.

#segmentation_dict = MD.try_segmentation() -- use this if you want to use the GUI to set parameters. May need it in an earlier cell.
segmentation_dict = {'model_name': ['2D_versatile_fluo'],
 'scale': 1.0,
 'prob_thresh': 0.7,
 'nms_thresh': 0.4,
 'vmin': 1.0,
 'vmax': 98.0}

failed = []
reg = False
nthreads = 16 #set this higher if you have threading enabled.  Marginal speed gains.

#this is significantly slower, but doesn't destroy data already processed if a position fails.  Failed position numbers are stored in the "failed" list.
if "driftTform" in MD().columns:
    reg = True
for i in positions_to_track:
    try:
        R.segment_and_extract_features(MD=MD,NucChannel=[nuc_mark],segment_type='stardist_nuclei' ,threads=nthreads, **segmentation_dict, register=reg, zernike=False,periring=True, Position = i)
    except:
        print(f"error at position {i}.")
        failed.append(i)
print(f'Positions that failed to segment: {failed}')

for i in positions_to_track:
    try:
        R.calculate_tracks(Position=i, params=[nuc_mark], search_radius=10,adaptive_radius=True, maxStep=5,minTrackLength=nframes//2,save=True, verbose = False)
    except:
        print(f'Failed position: {i}')

In [ ]:

#Set this value to the minimum number of frames for a track to be included in data output (default is nframes, for full tracks)
min_track_length = nframes -1

'''generates tracks of at least thresh length. Returns the useful tracks and their indices.
'''
def gen_full_tracks(track_ind_array, thresh = 1):
    useful_tracks = []
    ind_arr = []
    for ind, track in enumerate(track_ind_array):
        s = np.sum(track == None)
        if s <= thresh:
            useful_tracks.append(track)
            ind_arr.append(ind)
    return useful_tracks,ind_arr

#handling data and processing

#generate all relevant data -- this can potentially be edited, but be careful.
all_tracks = []
for pos in R.PosLbls.keys():
    g = R.PosLbls[pos]
    track_inds, track_labels = gen_full_tracks(g.trackinds, thresh = (nframes - min_track_length + 1))
    for ind, track in enumerate(track_inds):
        info_dict = {}
        for frame_ind, obj_id in enumerate(track):
            frame_dict = {}
            curr_frame = g.framelabels[frame_ind]
            next_frame = None
            if (frame_ind + 1 < len(track) and track[frame_ind + 1] != None):
                next_frame = g.framelabels[frame_ind+1]
            if obj_id == None:
                pass
            else:
                next_frame = g.framelabels[frame_ind+1]
                frame_dict["position"] = pos
                frame_dict["track_id"] = track_labels[ind]#ind -- updated for useful tracks
                frame_dict["frame_id"] = frame_ind
                frame_dict["object_id"] = obj_id
                frame_dict["weighted_centroid_x"] = curr_frame.regionprops['weighted_centroid'][obj_id][0]
                frame_dict["weighted_centroid_y"] = curr_frame.regionprops['weighted_centroid'][obj_id][1]
                if next_frame != None and track[frame_ind + 1] != None:
                    frame_dict["vel"] = math.dist(curr_frame.regionprops['centroid'][obj_id], next_frame.regionprops['centroid'][track[frame_ind+1]])
                    frame_dict["radius_delta"] = next_frame.regionprops['equivalent_diameter'][track[frame_ind + 1]]/2 - curr_frame.regionprops['equivalent_diameter'][obj_id]/2
                    frame_dict["area_delta"] = next_frame.regionprops['area'][track[frame_ind + 1]] - curr_frame.regionprops['area'][obj_id]
                    frame_dict["solidity_delta"] = next_frame.regionprops['solidity'][track[frame_ind + 1]] - curr_frame.regionprops['solidity'][obj_id]
                    #include this if you use phase.
                    #frame_dict["phase_delta"] = next_frame.regionprops['mean_Phase'][track[frame_ind + 1]] - curr_frame.regionprops['mean_Phase'][obj_id]
                    frame_dict["nuc_delta"] = next_frame.regionprops['mean_'+nuc_mark][track[frame_ind + 1]] - curr_frame.regionprops['mean_'+nuc_mark][obj_id]
                else:
                    frame_dict["vel"] = 0
                    frame_dict["radius_delta"] = 0
                    frame_dict["area_delta"] = 0
                    frame_dict["solidity_delta"] = 0
                frame_dict["area"] = curr_frame.regionprops['area'][obj_id]
                frame_dict["solidity"] = curr_frame.regionprops['solidity'][obj_id]
                frame_dict["radius"] = curr_frame.regionprops['equivalent_diameter'][obj_id]/2
                frame_dict["nuclear_marker"] = curr_frame.regionprops['mean_'+nuc_mark][obj_id]
                #include this if you use phase
                #frame_dict["phase"] = curr_frame.regionprops['mean_Phase'][obj_id] 
                for k in marker_labels.keys():
                    nuc = curr_frame.regionprops['mean_'+k][obj_id]
                    cyt = curr_frame.regionprops['mean_'+k+'_periring'][obj_id]
                    radius = curr_frame.regionprops['equivalent_diameter'][obj_id]/2
                    frame_dict[marker_labels[k]+"_nuc"] = nuc
                    frame_dict[marker_labels[k]+"_cyt"] = cyt
                    #generate ratios for all markers
                    area = curr_frame.regionprops['area'][obj_id]
                    peri_area = (radius + 5)**2 * np.pi - area
                    frame_dict[marker_labels[k]+"_ratio"] = nuc/(cyt + nuc)
                all_tracks.append(frame_dict)

#reformat and save to directory
all_tracks_dataframe = pd.DataFrame(all_tracks)
os.makedirs("./"+movie_name + "_analysis/csv_predict", exist_ok=True)
all_tracks_dataframe.to_csv(mpath + "/csv_predict/all_tracks_dataframe.csv")
#---------------- don't edit above here -------------#


## Analysis and graphing

In [ ]:

os.makedirs(mpath + "/graphs", exist_ok=True)
os.makedirs(mpath + "/formatted_data", exist_ok=True)
data = pd.read_csv(mpath + "/csv_predict/all_tracks_dataframe.csv")


### Quality Control

In [ ]:
#plot all cells area
plt.hist(data.area[data.area], bins = 100)
plt.show()

# The below code gets rid of cells under a certain size.  
# You can implement further cell track quality control here using the same ideas with different fields.
# bad_tracks = data[data.area < 200][['position', 'track_id']]
# g = bad_tracks.groupby(bad_tracks.columns.tolist(),as_index=False).size()
# g = g[g['size'] > 30]
# g = g.reset_index()

# #remove all data that doesn't fit qc standards (area, etc)
# for ind, pos in enumerate(g['position']):
#     data = data[~(np.array(data.position == pos) & np.array(data.track_id == g['track_id'][ind]))]
#     data = data[~(np.array(data.position == pos) & np.array(data.track_id == g['track_id'][ind]))]


The below blocks are older code that can be modified to show graphs across experimental conditions based on position.  They'll need some modification, but the idea is there.

### Multiple Position graphs

In [ ]:


lower = np.array(list(labels.values()))[:,0]
upper = np.array(list(labels.values()))[:,0]
data_to_analyze = data.copy()
all_foxo = []
all_foxo_std = []
all_p53 = []
all_p53_std = []
for i in range(len(lower)):
    l = data_to_analyze.position >= lower[i]
    u = data_to_analyze.position <= upper[i]
    both = l & u
    pos_subset = data_to_analyze[both]
    foxo_ratio = []
    foxo_std = []
    p53_nuc = []
    p53_std = []
    for i in range(len(set(pos_subset.frame_id))):
        data_subset = pos_subset[pos_subset.frame_id == i]
        foxo_ratio.append(np.mean(data_subset.red_ratio))
        foxo_std.append(np.std(data_subset.red_ratio)/np.sqrt(len(data_subset.red_ratio)))
        p53_nuc.append(np.mean(data_subset.blue_nuc))
        p53_std.append(np.std(data_subset.blue_nuc)/np.sqrt(len(data_subset.blue_nuc)))
    all_foxo.append(np.array(foxo_ratio))
    all_p53.append(np.array(p53_nuc))
    all_foxo_std.append(np.array(foxo_std))
    all_p53_std.append(np.array(p53_std))



In [ ]:
data

In [ ]:
data_to_analyze = data.copy()
all_foxo = []
all_foxo_std = []
all_p53 = []
all_p53_std = []
for i in range(len(lower)):
    l = data_to_analyze.position >= lower[i]*10
    u = data_to_analyze.position < upper[i]*10
    both = l & u
    pos_subset = data_to_analyze[both]
    foxo_ratio = []
    foxo_std = []
    p53_nuc = []
    p53_std = []
    for i in range(len(set(pos_subset.frame_id))):
        data_subset = pos_subset[pos_subset.frame_id == i]
        foxo_ratio.append(np.mean(data_subset.yfp_ratio))
        foxo_std.append(np.std(data_subset.yfp_ratio)/np.sqrt(len(data_subset.yfp_ratio)))
        p53_nuc.append(np.mean(data_subset.p53_nuc))
        p53_std.append(np.std(data_subset.p53_nuc)/np.sqrt(len(data_subset.p53_nuc)))
    all_foxo.append(np.array(foxo_ratio))
    all_p53.append(np.array(p53_nuc))
    all_foxo_std.append(np.array(foxo_std))
    all_p53_std.append(np.array(p53_std))


In [ ]:
print(list(labels.keys()))
nframes = 255

In [ ]:
os.makedirs(fpath + '/graphs/experiments', exist_ok=True)
plt.figure(figsize=(8,6))
for i in range(len(all_foxo)):
    fig, graph = plt.subplots()
    t = np.linspace(.25,(nframes)//12, nframes)
    l1 = graph.plot(t, all_foxo[i], color = 'red', label = "Mean blue ratio sig")
    ci_lower_f = all_foxo[i] - all_foxo_std[i] * 2
    ci_upper_f = all_foxo[i] + all_foxo_std[i] * 2
    l2 = graph.fill_between(t, ci_lower_f, ci_upper_f, color='red', alpha=.15, label = "blue%CI")
    graph.set_ylabel('blue ratio', color = 'blue')
    graph.tick_params(axis = 'y', labelcolor = 'blue')
    graph.set_ylim(0.5, 2.5)
    graph.xaxis.set_ticks(np.arange(0, 8, 1))
    plt.legend(["blue ratio", "blue%CI"], loc = 2)

    g2 = graph.twinx()
    l3 = g2.plot(t, all_p53[i], label = "nuc sig")
    ci_lower_p = all_p53[i] - all_p53_std[i] * 2
    ci_upper_p = all_p53[i] + all_p53_std[i] * 2
    l4 = g2.fill_between(t, ci_lower_p, ci_upper_p, color='b', alpha=.15, label = "nuc%CI")
    g2.set_ylabel('nuc', color = 'b')
    g2.tick_params(axis = 'y', labelcolor = 'b')
    g2.set_ylim(0, 1)
    fig.tight_layout() 


    plt.title(f'Positions {lower[i] + 1} to {upper[i]} -- {list(labels.keys())[i]}')
    plt.xlim(0, 8)
    #plt.legend(["Mean h3 ratio", "h3_95%CI"], loc = 4)

    #plt.savefig(path + '/graphs/experiments/pos'+str(lower[i] +1)+'_'+str(upper[i])+'_combined.png')
    plt.show()


In [ ]:
plt.show()

In [ ]:
#p53 or foxo graphs
lab = list(labels.keys())
os.makedirs(mpath + "/graphs/experiments/p53", exist_ok = True)
os.makedirs(mpath + "/graphs/experiments/foxo", exist_ok = True)
foxo_g = False
flab = ""
nframes = 145
if (foxo_g):
    flab = "foxo"
else:
    flab = "p53"
for i in range(len(all_foxo)):
    fig, graph = plt.subplots()
    t = np.linspace(0,nframes//3, nframes-1)
    if foxo_g:
        graph.plot(t, all_foxo[i], color = 'red')
        ci_lower_f = all_foxo[i] - all_foxo_std[i] * 2
        ci_upper_f = all_foxo[i] + all_foxo_std[i] * 2
        graph.fill_between(t, ci_lower_f, ci_upper_f, color='red', alpha=.15)
        plt.legend(["Mean Foxo Nuc-Cyto Ratio", "f_ratio_95%CI"])

    else:
        graph.plot(t, all_p53[i])
        ci_lower_p = all_p53[i] - all_p53_std[i] * 2
        ci_upper_p = all_p53[i] + all_p53_std[i] * 2
        graph.fill_between(t, ci_lower_p, ci_upper_p, color='b', alpha=.15)
        plt.legend(["Mean p53 Nuclear Signal", "p53_95%CI"])

    plt.title(f'Positions {lower[i] + 1} to {upper[i]} -- {lab[i]}')
    plt.xlim(0, 48)
    #plt.ylim(0, .65)
    plt.savefig(path + '/graphs/experiments/'+flab+'/pos'+str(lower[i] +1)+'_'+str(upper[i]+1)+'_'+flab+'.png')
    plt.show()


### Single Position Graphs

In [ ]:
#for single positions
positions_to_track = []
for i in labels.values():
    positions_to_track = positions_to_track + list(i)
positions_to_track = [str(i) for i in positions_to_track]
data_to_analyze = data.copy()
all_foxo = []
all_foxo_std = []
all_p53 = []
all_p53_std = []
for pos in positions_to_track:
    both = data_to_analyze.position == int(pos)
    pos_subset = data_to_analyze[both]
    foxo_ratio = []
    foxo_std = []
    p53_nuc = []
    p53_std = []
    for i in range(len(set(pos_subset.frame_id))):
        data_subset = pos_subset[pos_subset.frame_id == i]
        foxo_ratio.append(np.mean(data_subset.red_ratio))
        foxo_std.append(np.std(data_subset.red_ratio)/np.sqrt(len(data_subset.red_ratio)))
        p53_nuc.append(np.mean(data_subset.blue_nuc))
        p53_std.append(np.std(data_subset.blue_nuc)/np.sqrt(len(data_subset.blue_nuc)))
    all_foxo.append(np.array(foxo_ratio))
    all_p53.append(np.array(p53_nuc))
    all_foxo_std.append(np.array(foxo_std))
    all_p53_std.append(np.array(p53_std))

In [ ]:
#single positions
labels = ["0"]
os.makedirs(fpath + "/graphs/single_positions/combined", exist_ok = True)
#labels = ["exp1", "exp2", "exp3"]
for i in range(len(all_foxo)):
    fig, graph = plt.subplots()
    t = np.linspace(0,len(all_foxo[i])//4, len(all_foxo[i]))
    graph.plot(t, all_foxo[i], color = 'red')
    ci_lower_f = all_foxo[i] - all_foxo_std[i] * 2
    ci_upper_f = all_foxo[i] + all_foxo_std[i] * 2
    graph.fill_between(t, ci_lower_f, ci_upper_f, color='red', alpha=.15)
    graph.plot(t, all_p53[i])
    ci_lower_p = all_p53[i] - all_p53_std[i] * 2
    ci_upper_p = all_p53[i] + all_p53_std[i] * 2
    graph.fill_between(t, ci_lower_p, ci_upper_p, color='b', alpha=.15)
    plt.title(f'Position {lower[i]} -- {labels[i//10]}')
    plt.xlim(0, 24)
    plt.legend(["Mean Foxo Nuc-Cyto Ratio", "f_ratio_95%CI", "Mean p53 Nuclear Signal", "p53_95%CI"])
    #plt.ylim(0, 1)
    plt.savefig(fpath + '/graphs/single_positions/combined/pos'+str(lower[i])+'_combined.png')
    plt.show()

In [ ]:
#single positions foxo or p53
#labels = ["2", "1", "0.75", "0.5", "0.05", "0.1", "0.25", "0.01", "UT"]
os.makedirs(path + "/graphs/single_positions/p53", exist_ok = True)
os.makedirs(path + "/graphs/single_positions/foxo", exist_ok = True)
foxo_g = True
flab = ""
if (foxo_g):
    flab = "foxo"
else:
    flab = "p53"
#labels = ["exp1", "exp2", "exp3"]
for i in range(len(all_foxo)):
    fig, graph = plt.subplots()
    t = np.linspace(0,len(all_foxo[i])//3, len(all_foxo[i]))
    if (foxo_g):
        graph.plot(t, all_foxo[i], color = 'red')
        ci_lower_f = all_foxo[i] - all_foxo_std[i] * 2
        ci_upper_f = all_foxo[i] + all_foxo_std[i] * 2
        graph.fill_between(t, ci_lower_f, ci_upper_f, color='red', alpha=.15)
        plt.legend(["Mean Foxo Nuc-Cyto Ratio", "f_ratio_95%CI"])

    else:
        graph.plot(t, all_p53[i])
        ci_lower_p = all_p53[i] - all_p53_std[i] * 2
        ci_upper_p = all_p53[i] + all_p53_std[i] * 2
        graph.fill_between(t, ci_lower_p, ci_upper_p, color='b', alpha=.15)
        plt.legend(["Mean p53 Nuclear Signal", "p53_95%CI"])

    plt.title(f'Position {positions_to_track[i]} -- {list(labels.keys())[i//10]}')
    plt.xlim(0, 96)
    plt.ylim(.4, .75)
    plt.savefig(path + '/graphs/single_positions/'+flab+'/pos'+positions_to_track[i]+'_'+flab+'.png')
    plt.show()

### Single Track Graphs

In [ ]:
position = np.array(list(labels.values()))[:,0] #choose tracks from each of 10 positions
#position = np.array([0, 1, 2])
data_to_analyze = data.copy()
all_foxo = []
all_p53 = []
n_tracks_per_pos = 10
for i, val in enumerate(position):
    pos = data_to_analyze.position == val
    pos_subset = data_to_analyze[pos]
    track_list = np.array(list(set(pos_subset.track_id)))
    tracks_to_plot = np.random.choice(track_list, min(len(track_list), n_tracks_per_pos), replace = False)
    for j, track in enumerate(tracks_to_plot):
        data_subset = pos_subset[pos_subset.track_id == track]
        all_foxo.append([(val, track),np.array(data_subset.blue_ratio)])
        all_p53.append([(val, track),np.array(data_subset.red_nuc)])

In [ ]:
#graphing single-cell trajectories
#labels = ["exp1", "exp2", "exp3"]
lab = list(labels.keys())
os.makedirs(mpath + "/graphs/single_tracks/combined", exist_ok=True)

for i in range(len(all_foxo)):
    fig, graph = plt.subplots()
    curr_data_f = all_foxo[i]
    curr_data_p = all_p53[i]
    pos = curr_data_f[0][0]
    track = curr_data_f[0][1]
    t = np.linspace(0,len(curr_data_f[1])//4, len(curr_data_f[1]))
    graph.plot(t, curr_data_f[1], color = 'red')
    graph.plot(t, curr_data_p[1])
    plt.title(f'Position {pos}, Track {track}-- {lab[pos//10]}')
    plt.xlim(0, 64)
    plt.legend(["Foxo Nuc-Cyto Ratio", "p53 Nuclear Signal"])
    #plt.ylim(0, 1)
    plt.savefig(mpath + '/graphs/single_tracks/combined/pos'+str(pos)+'_track'+str(track)+'_combined.png')
    plt.show()

In [ ]:
#graphing single tracks foxo or p53

lab = list(labels.keys())
os.makedirs(mpath + "/graphs/single_tracks/p53", exist_ok = True)
os.makedirs(mpath + "/graphs/single_tracks/foxo", exist_ok = True)
foxo_g = True
flab = ""
if (foxo_g):
    flab = "foxo"
else:
    flab = "p53"
for i in range(len(all_foxo)):
    fig, graph = plt.subplots()
    curr_data_f = all_foxo[i]
    #curr_data_p = all_p53[i]
    pos = curr_data_f[0][0]
    track = curr_data_f[0][1]
    t = np.linspace(0,len(curr_data_f[1])//3, len(curr_data_f[1]))
    if foxo_g:
        graph.plot(t, curr_data_f[1], color = 'red')
        plt.legend(["Foxo Nuclear Fraction"])
    else:
        graph.plot(t, curr_data_p[1])
        plt.legend(["p53 Nuclear Signal"])

    plt.title(f'Position {pos}, Track {track}-- {list(labels.keys())[i//10]}')
    plt.xlim(0, 24)
    plt.ylim(0.35, .8)
    plt.savefig(mpath + '/graphs/single_tracks/'+flab+'/pos'+str(pos)+'_track'+str(track)+'_'+flab+'.png')
    plt.show()

### Heatmaps

In [ ]:
#generating heatmaps from all single-cell traces. -- This is older and not necessarily useful anymore.
#use nan data.

def generate_heatmap_array(data, timepoints, field, positions = None):
    num_pos = list(set(data.position))
    if positions != None:
        num_pos = positions
    num_rows = 0
    for i in range(len(num_pos)):
        num_tracks = list(set(data[data.position == num_pos[i]].track_id))
        num_rows += len(num_tracks)
    heatmap_array = np.empty((num_rows, timepoints))
    row_count = 0
    for i in range(len(num_pos)):
        pos_data = data[data.position == num_pos[i]]
        for id, track in enumerate(list(set(pos_data.track_id))):
            track_data = pos_data[pos_data.track_id == track]
            for row in range(len(track_data)):
                fr = track_data.iloc[row,:]
                frame_id = fr.frame_id
                heatmap_array[row_count, int(frame_id)] = fr[field]
            row_count +=1
    return heatmap_array



In [ ]:
#smoothing function, identify width, default is to normalize data prior to smoothing, window width set to 5.
def smooth(data, width = 5, norm = True):
    data_norm = data.copy()
    if norm:
        data_norm = (data - np.mean(data))/np.std(data)
    cumsum_vec = np.cumsum(np.insert(data_norm, 0, 0)) 
    ma_vec = (cumsum_vec[width:] - cumsum_vec[:-width]) / width
    return ma_vec

In [ ]:
from scipy.signal import find_peaks
div_counts = []
channel = "red_nuc"
channel2 = "blue_ratio"
position = np.array(list(labels.values()))[:,0] 
#position = np.array([0, 1, 2])
data_to_analyze = data.copy()
for i, val in enumerate(position):
    pos = data_to_analyze.position == val
    pos_subset = data_to_analyze[pos]
    track_list = np.array(list(set(pos_subset.track_id)))
    for j, track in enumerate(track_list):
        data_subset = pos_subset[pos_subset.track_id == track]
        thresh = .08
        dat = smooth(data_subset[channel],4) + smooth(data_subset[channel2], 4)
        dat_e = np.exp(dat)/sum(np.exp(dat))
        div_counts.append([(val, track),
                           find_peaks(dat_e, distance = 45,prominence = thresh)])
        #all_p53.append([(val, track),np.array(data_subset.p53_nuc)])

In [ ]:
tr = 68
#plt.plot(t,smooth(data_to_analyze[data_to_analyze.track_id == tr].red_nuc))
#plt.plot(t,smooth(data_to_analyze[data_to_analyze.track_id == tr].blue_ratio))
for i in list(set(data_to_analyze.track_id)):
    t = range(len(data_to_analyze[data_to_analyze.track_id == i].blue_ratio)-4)
    dat = smooth(data_to_analyze[data_to_analyze.track_id == i].blue_ratio) + smooth(data_to_analyze[data_to_analyze.track_id == i].red_nuc)
    dat_e = np.exp(dat)/sum(np.exp(dat))
    plt.plot(t,dat_e)

In [ ]:
tr = 68
#plt.plot(t,smooth(data_to_analyze[data_to_analyze.track_id == tr].red_nuc))
#plt.plot(t,smooth(data_to_analyze[data_to_analyze.track_id == tr].blue_ratio))
t = range(len(data_to_analyze[data_to_analyze.track_id == tr].blue_ratio)-4)
dat = smooth(data_to_analyze[data_to_analyze.track_id == tr].blue_ratio) + smooth(data_to_analyze[data_to_analyze.track_id == tr].red_nuc)
dat_e = np.exp(dat)/sum(np.exp(dat))
plt.plot(t[10:50],dat[10:50])
plt.plot(t,dat)

In [ ]:
tr = 27
#plt.plot(t,smooth(data_to_analyze[data_to_analyze.track_id == tr].red_nuc, norm = False))
plt.plot(t,smooth(data_to_analyze[data_to_analyze.track_id == tr].blue_ratio, norm = False))
plt.show()

In [ ]:
plt.hist(np.exp(1- data_to_analyze.blue_ratio)/np.sum(np.exp(1- data_to_analyze.blue_ratio)), bins = 25)
plt.show()

In [ ]:
#Generates opposite ratio for C/N data, binarizes based on threshold (Active True vs. inactive False)
data["c_n_ratio"] = 1 - data.blue_ratio
data["c_n_binary"] = [int(i) for i in (data["c_n_ratio"] > 0.50)]


In [ ]:
#plt.plot(data_to_analyze[data_to_analyze.track_id == 27].c_n_ratio)
val = 20
plt.plot(np.round(smooth(data[data.track_id == val].c_n_binary, norm = False, width = 8)))
plt.plot(smooth(data[data.track_id == val].c_n_ratio, norm = False))
plt.plot(smooth(data[data.track_id == val].red_nuc, norm = False))

In [ ]:
#for inspection of data.
val = 2
k = data[data.track_id == val].red_nuc
g = smooth(k, norm = False) 
gdiff = g[1:] - g[:-1]
gdiff2 = gdiff[1:] - gdiff[:-1]
norm_gdiff = (gdiff - np.mean(gdiff))/np.std(gdiff)

#plt.plot(norm_gdiff >=4)
#plt.plot(g[1:] - g[:-1])
#plt.plot(lev)
print(np.argmax(k[1:]))
print(np.argmax(g))
plt.plot(range(len(k)-1), k[1:])
plt.plot(range(len(g)), g)

### This section and the next are important for heatmap ordering

In [ ]:
#----------------------condensed version of code----------------------------#

#import the regular expression package
import re

#unique track ids in data -- you'd need to subset data if you have multiple positions, but this works if you have one position
track_list = list(set(data.track_id))
track_peaks = [] #used for downstream analysis -- will have a list of all timepoints above a 4.2 SD threshold in the nuclear channel velocity.
#track_peaks stores arrays of the format [track_id, peak_list] for each track.

#move through all tracks
for ind, track in enumerate(track_list):
    #smooth nuclear data and store the delta nuc values (velocity values) in gdiff, then standardize
    g = smooth(data[data.track_id == track].red_nuc, norm = False) 
    gdiff = g[1:] - g[:-1] #note, this subtracts 1 from the real index when applied.
    norm_gdiff = (gdiff - np.mean(gdiff))/np.std(gdiff)

    #Now analyze
    where_split = np.where(norm_gdiff >=4.2) #where are velocity peaks greater than 4.2 standard deviations above the mean? using this
    smoothed_binary = [int(round(i)) for i in smooth(data[data.track_id == track].c_n_binary, norm = False)] #smoothed binary to find transitions.
    stringified_bin = ''.join(map(str, smoothed_binary)) #convert to string
    already_on = stringified_bin.find('1') #using this to find on values.
    if already_on == -1: # if there are no CDK2 on events
        track_peaks.append([track, []]) #add no peaks for that track
    else:
        track_peaks.append([track, where_split[0] + 1]) #add peaks 1 later due to velocity conversion. -- where_split[0] takes the position list from np.where.


In [ ]:
#this section does the heavy lifting related to determining division events and timing.  Not all fields are used. -- old version
import re

track_list = list(set(data.track_id))
track_vals = np.zeros((len(track_list), 5))
track_peaks = []
track_peaks2 = []
for ind, track in enumerate(track_list):
    g = smooth(data[data.track_id == track].red_nuc, norm = False) 
    gdiff = g[1:] - g[:-1]
    norm_gdiff = (gdiff - np.mean(gdiff))/np.std(gdiff)
    where_split = np.where(norm_gdiff >=4.2) #where are velocity peaks greater than 4.2 standard deviations above the mean? using this
    smoothed_binary = [int(round(i)) for i in smooth(data[data.track_id == track].c_n_binary, norm = False)] #smoothed binary to find transitions.
    stringified_bin = ''.join(map(str, smoothed_binary))
    first_off = stringified_bin.find('10')
    first_on = stringified_bin.find('01')
    already_on = stringified_bin.find('1') #using this
    count_div = len([m.start() for m in re.finditer('10', stringified_bin)]) #use string finder to count transitions from on to off.
    if already_on == -1:
        track_peaks.append([track, []]) #if no CDK transition, add no peaks
    else:
        track_peaks.append([track, where_split[0] + 1]) #add peak 1 later due to velocity conversion.
        track_peaks2.append([track, np.where(g -np.min(g) >=0.05)]) #Alternate method for calculating peaks.
    track_vals[ind,0] = first_off
    track_vals[ind,1] = first_on
    track_vals[ind,2] = already_on
    track_vals[ind,3] = count_div
    track_vals[ind,4] = track
    

In [ ]:
#goal -- find max value in interval of three to each side around these elements.
#Order of events:
#If peaks exist, find the biggest value in data for each found peak, append to location list.
#This gets rid of duplicate events where multiple points in a sequence are above the 4.2 threshold.
#Count elements to determine number of divisions.
count_arr = np.zeros((len(track_peaks), 3)) #for storage, format is [track_id, num_divisions, time_of_first_division]
div_arr = []
for ind, row in enumerate(track_peaks):
    track = row[0] #track id
    vals = row[1] #potential peak positions
    count_arr[ind, 0] = track
    g = smooth(data[data.track_id == track].red_nuc, norm = False)  #smoothed nuclear marker data, non-normalized
    g = g - np.min(g) #leveled data (starts from zero)
    if len(vals) != 0:
        temp_arr = []
        for val in vals:
            temp_arr.append(np.max(g[val - 3:val + 6])) #store the maximum value in a window centered around the current potential peak
        time_first = np.where(g == temp_arr[0])[0] #find the location in nuc of the first peak
        div_arr.append([track, [np.where(g == r)[0][0] + 1 for r in list(set(temp_arr))]]) #stores the location of all true divisions -- not used here
        count_arr[ind, 1] = len(list(set(temp_arr))) #counts number of peaks
        count_arr[ind, 2] = time_first[0] + 1 #stores first location of first peak.
    else:
        div_arr.append([track, []]) #if no peaks, add empty list


In [ ]:
#develop ordering by time of first division.
df = pd.DataFrame(count_arr, dtype='int')
df.columns = ["track", "divs", "first_div"]
heatmap_ordering = df.sort_values(by = ['divs', 'first_div'], ascending = [False, True]) #generate ordering based on divs, then time to first division.

In [ ]:
nframes = 256
interval = 0.25 #interval between frames, in hours
hm_marker = "c_n_ratio"
#marker heatmap data
heatmap_data = np.zeros((len(heatmap_ordering), nframes)) #make a rectangular array of number of tracks by number of frames
for ind, track in enumerate(heatmap_ordering.track): #for each track in order
    sub_data = data[data.track_id == track] #find the data from that track
    min_frame = np.min(sub_data.frame_id) #find the minimum frame id from that track

    #store the data from the desired hm_marker in the proper place in the heatmap array, starting from min_frame
    heatmap_data[ind, min_frame:min_frame + len(sub_data[hm_marker])] = sub_data[hm_marker] 

hm_marker = "red_nuc"
heatmap_data2 = np.zeros((len(heatmap_ordering), nframes))
for ind, track in enumerate(heatmap_ordering.track):
    sub_data = data[data.track_id == track]
    min_frame = np.min(sub_data.frame_id)
    heatmap_data2[ind, min_frame:min_frame + len(sub_data[hm_marker])] = sub_data[hm_marker]

#division heatmap data
heatmap_divs = np.zeros((len(heatmap_ordering), nframes))
for ind, index in enumerate(heatmap_ordering.index):
    for val in div_arr[index][1]:
        heatmap_divs[ind,val:] +=1
times = np.linspace(0, len(heatmap_data[0])-1, nframes)*interval
df_hm_data = pd.DataFrame(heatmap_data, columns = times)
df_hm_data2 = pd.DataFrame(heatmap_data2, columns = times)
df_hm_div = pd.DataFrame(heatmap_divs, columns = times)

In [ ]:
os.makedirs(mpath + "/graphs/experiments/heatmaps", exist_ok=True)
plt.rcParams["figure.figsize"] = 6,6

r = sns.heatmap(df_hm_data, cmap = "viridis", vmin = 0.4, annot = False) #use the dataframe with ordered heatmap data, choose minima and maxima wisely
r.set_facecolor('gray')
plt.title("CDK2 activity")
plt.xlabel("Time (hrs)")
plt.ylabel("Cell trace")
plt.savefig(mpath + "/graphs/experiments/heatmaps/CDK_activity.svg")
plt.show()


In [ ]:
os.makedirs(mpath + "/graphs/experiments/heatmaps", exist_ok=True)
plt.rcParams["figure.figsize"] = 6,6

r = sns.heatmap(df_hm_data2, cmap = "viridis", vmin = 0.06, annot = False)
r.set_facecolor('gray')
plt.title("Nuclear marker intensity")
plt.xlabel("Time (hrs)")
plt.ylabel("Cell trace")
plt.savefig(mpath + "/graphs/experiments/heatmaps/red_nuc.svg")
plt.show()


In [ ]:
g = sns.heatmap(df_hm_div, cmap = "viridis")
plt.title("Number of Divisions")
plt.xlabel("Time (hrs)")
plt.ylabel("Cell trace")
plt.savefig(mpath + "/graphs/experiments/heatmaps/divs_hm.svg")
plt.show()

#### Heatmap example for FOXO/p53 data -- edit as desired.
This is an approach that works for combining cells across multiple positions from the same experiment, detailed in the *labels.csv* 

In [ ]:
all_matrices = []
lower = np.array(list(labels.values()))[:,0]
upper = np.array(list(labels.values()))[:,9]
nframes = 144
for i in range(len(lower)):
    r = range(lower[i], upper[i]+1)
    if (upper[i] == 139):
        r =  range(lower[i], upper[i])
    foxo_heatmap_values = generate_heatmap_array(data, nframes, "foxo_ratio", positions = r)
    p53_heatmap_values = generate_heatmap_array(data, nframes, "p53_nuc", positions = r)
    all_matrices.append([foxo_heatmap_values,p53_heatmap_values])#, p53_heatmap_values])


In [ ]:
names = list(labels.keys())
#names = ["E1", "E2", "E3"]
#names = ["E1"]
os.makedirs(mpath + "/graphs/experiments/heatmaps", exist_ok=True)
for i, data in enumerate(excluded):
    colormap_input = generate_colormap_data(data[0])
    colormap_input = np.array(colormap_input)
    y = np.linspace(0,1,len(colormap_input))
    print(np.sum(colormap_input)/len(colormap_input))
    ext = [y[0]-(y[1]-y[0])/2., y[-1]+(y[1]-y[0])/2.,len(colormap_input),0]
    plt.rcParams["figure.figsize"] = 2,5
    plt.imshow(colormap_input[:,np.newaxis], cmap = 'Greys', aspect = "auto", extent = ext)
    plt.show()
    g = sns.heatmap(data[1], cmap = "viridis", vmin = .03, vmax = 0.06)
    g.set_facecolor('gray')
    plt.title("p53 traces, "+names[i])
    plt.savefig(mpath + "/graphs/experiments/heatmaps/p53_"+names[i]+".svg")
    plt.show()
    #scale = np.nanpercentile(data[1], 99)
    #print(scale)
    #scale_l = np.nanpercentile(data[1], 10)
    #g = sns.heatmap(data[1], cmap = "viridis", vmin = scale_l, vmax = scale)
    #g.set_facecolor('gray')
    #plt.title("p53 traces, "+names[i])
    #plt.savefig(path + "/graphs/experiments/heatmaps/trial/p53_"+names[i]+".png")
    #plt.show()
#plt.rcParams["figure.figsize"] = 5,2

#x = np.linspace(-3,3)
#y = np.cumsum(np.random.randn(50))+6

# fig, (ax,ax2) = plt.subplots(nrows=2, sharex=True)

# extent = [x[0]-(x[1]-x[0])/2., x[-1]+(x[1]-x[0])/2.,0,1]
# ax.imshow(y[np.newaxis,:], cmap="plasma", aspect="auto", extent=extent)
# ax.set_yticks([])
# ax.set_xlim(extent[0], extent[1])

#plt.tight_layout()
#plt.show()

## Load saved results object for further exploration

In [ ]:
R = pd.read_pickle(fpath + "/results.pickle")

In [ ]:
#visualization in napari.
from oyLabImaging.Processing.imvisutils import get_or_create_viewer

viewer = get_or_create_viewer()

viewer.layers.clear()
position_to_view = '0'
R.show_images(pos=str(position_to_view), Channel=[nuc_mark],cmaps=['red'])
R.show_images(pos = str(position_to_view), Channel = [cyto_mark], cmaps = ['green'])
R.show_tracks(pos=str(position_to_view))
R.track_explorer()



## Movie Generation

In [ ]:
#requires a computer with functional napari viewer
from oyLabImaging.Processing.imvisutils import get_or_create_viewer
from oyLabImaging.Processing.imvisutils import export_napari_to_movie
import oyLabImaging
#might crash if you generate too many movies.
#if you want to include other visual elements, you'll have to add new functionality to/consult the Results object in the oyLabImaging package
positionlist = R.PosLbls.keys()
viewer = get_or_create_viewer()
for i in positionlist:
    viewer.layers.clear()
    position_to_view = i
    g = R.PosLbls[i]
    track_inds, track_labels = gen_full_tracks(g.trackinds)
    R.show_images(pos=str(position_to_view), Channel=[nuc_mark],cmaps=['gray'])
    R.show_tracks(pos=str(position_to_view), J = track_labels)
    #take movie snapshot of current napari window.
    os.makedirs(mpath + "/processed_movie",exist_ok=True)
    movie_label = "pos_"+str(i)+"_tracks"
    export_napari_to_movie(fname=mpath + '/processed_movie/Napari_movie_'+movie_name+'_'+movie_label+'.mov')
